<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/Fast_Random_Fade_Predictorv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Script with Material Selection, Environmental Parameter Inputs,
UV/Visible Light Exposure Simulation, Color Fading Prediction,
Delta-E Calculations, Average Color, and Histograms with Heatmaps.
"""

import sys
import subprocess
import importlib
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Function to install packages
def install_packages(packages):
    for package in packages:
        if package == 'google.colab':
            continue  # Skip installation as it's already available in Colab
        try:
            importlib.import_module(package)
            logging.info(f"Package '{package}' is already installed.")
        except ImportError:
            print(f"Installing package: {package}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    'scikit-image', 'numpy', 'pandas', 'matplotlib',
    'Pillow', 'scipy', 'sklearn', 'ipywidgets'
]
install_packages(required_packages)

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from skimage import color
from skimage.color import deltaE_ciede2000  # Import deltaE_ciede2000
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets

# For file uploads and downloads in Colab
try:
    from google.colab import files
except ImportError:
    # Define dummy functions if not in Colab
    class DummyFiles:
        def upload(self):
            print("File upload functionality is not implemented in this environment.")
            return {}
        def download(self, filename):
            print(f"File download functionality is not implemented. File '{filename}' saved locally.")
    files = DummyFiles()

# -----------------------------
# Step 1: Data Upload and Preprocessing
# -----------------------------

def upload_file(prompt_message, file_types=None):
    print(prompt_message)
    uploaded = files.upload()
    if not uploaded:
        logging.error("No file uploaded.")
        sys.exit(1)
    filename = next(iter(uploaded))
    if file_types and not filename.lower().endswith(file_types):
        logging.error(f"Uploaded file must be one of the following types: {file_types}")
        sys.exit(1)
    logging.info(f"Uploaded file: {filename}")
    return filename

def load_and_clean_dataset(csv_filename):
    try:
        dataset = pd.read_csv(csv_filename)
        required_columns = {'L', 'A', 'B', 'Color Name'}
        if not required_columns.issubset(dataset.columns):
            missing = required_columns - set(dataset.columns)
            logging.error(f"Dataset is missing required columns: {missing}")
            sys.exit(1)
        dataset = dataset.replace([np.inf, -np.inf], np.nan).dropna(subset=['L', 'A', 'B'])
        logging.info(f"Dataset loaded with {len(dataset)} entries after cleaning.")
        return dataset
    except Exception as e:
        logging.error(f"Failed to load dataset: {e}")
        sys.exit(1)

def upload_and_process_image():
    image_filename = upload_file("Please upload the image file you want to analyze.", file_types=('.png', '.jpg', '.jpeg'))
    try:
        image = Image.open(image_filename).convert('RGB')
        image_array = np.array(image).astype(np.float32) / 255.0
        lab_image = color.rgb2lab(image_array)
        logging.info(f"Image '{image_filename}' loaded and converted to LAB color space.")
        return image, lab_image
    except Exception as e:
        logging.error(f"Failed to process image: {e}")
        sys.exit(1)

# -----------------------------
# Step 2: Machine Learning Model Training
# -----------------------------

def create_synthetic_data(materials, num_samples_per_material=1000, random_seed=42):
    np.random.seed(random_seed)
    data_list = []
    for material in materials:
        # Generate synthetic fading data based on material properties
        if material == 'Acidic Paper':
            # Acidic paper tends to yellow and darken over time
            L_fading = np.random.normal(loc=-5, scale=2, size=num_samples_per_material)
            A_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
            B_fading = np.random.normal(loc=5, scale=2, size=num_samples_per_material)
        elif material == 'Alkaline Paper':
            # Alkaline paper is more stable
            L_fading = np.random.normal(loc=-2, scale=1, size=num_samples_per_material)
            A_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
            B_fading = np.random.normal(loc=1, scale=1, size=num_samples_per_material)
        elif material == 'Chromolithograph Print':
            # Vibrant colors that may fade under UV exposure
            L_fading = np.random.normal(loc=-3, scale=1.5, size=num_samples_per_material)
            A_fading = np.random.normal(loc=-2, scale=1, size=num_samples_per_material)
            B_fading = np.random.normal(loc=-2, scale=1, size=num_samples_per_material)
        elif material == 'Sanguine Etching':
            # Red pigments that may fade with light exposure
            L_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
            A_fading = np.random.normal(loc=-3, scale=1.5, size=num_samples_per_material)
            B_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
        elif material == 'Steel Engraving':
            # Generally stable but may darken slightly
            L_fading = np.random.normal(loc=-1, scale=0.5, size=num_samples_per_material)
            A_fading = np.random.normal(loc=0, scale=0.5, size=num_samples_per_material)
            B_fading = np.random.normal(loc=0, scale=0.5, size=num_samples_per_material)
        elif material == 'Wove Paper':
            # May yellow over time
            L_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
            A_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
            B_fading = np.random.normal(loc=3, scale=1, size=num_samples_per_material)
        elif material == 'Rag Paper':
            # Durable but can yellow slightly
            L_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
            A_fading = np.random.normal(loc=0, scale=1, size=num_samples_per_material)
            B_fading = np.random.normal(loc=2, scale=1, size=num_samples_per_material)
        else:
            # Default synthetic data
            L_fading = np.random.normal(loc=0, scale=5, size=num_samples_per_material)
            A_fading = np.random.normal(loc=0, scale=2, size=num_samples_per_material)
            B_fading = np.random.normal(loc=0, scale=2, size=num_samples_per_material)

        lux_hours = np.random.uniform(low=1000, high=100000, size=num_samples_per_material)
        material_array = np.array([material]*num_samples_per_material)

        data = pd.DataFrame({
            'material': material_array,
            'lux_hours': lux_hours,
            'L_fading': L_fading,
            'A_fading': A_fading,
            'B_fading': B_fading
        })
        data_list.append(data)
    synthetic_data = pd.concat(data_list, ignore_index=True)
    logging.info(f"Synthetic data generated for materials: {materials}")
    return synthetic_data

def prepare_features(synthetic_data):
    X_numeric = synthetic_data[['lux_hours']]
    X_categorical = synthetic_data[['material']]

    # One-hot encode the material types
    encoder = OneHotEncoder(sparse=False)
    X_material_encoded = encoder.fit_transform(X_categorical)

    # Combine numeric and encoded categorical features
    X = np.hstack((X_numeric.values, X_material_encoded))
    Y = synthetic_data[['L_fading', 'A_fading', 'B_fading']].values
    return X, Y, encoder

def train_ml_model(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rf_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
    rf_model.fit(X_train_scaled, Y_train)
    logging.info("Random Forest model trained.")
    Y_pred = rf_model.predict(X_test_scaled)
    mse = mean_squared_error(Y_test, Y_pred)
    logging.info(f"Mean Squared Error for Fading Prediction: {mse:.4f}")
    return rf_model, scaler, mse

# -----------------------------
# Step 3: Material Selection and Exposure Simulation Functions
# -----------------------------

# Update material options
material_dropdown = widgets.Dropdown(
    options=[
        'Acidic Paper',
        'Alkaline Paper',
        'Chromolithograph Print',
        'Sanguine Etching',
        'Steel Engraving',
        'Wove Paper',
        'Rag Paper',
    ],
    value='Acidic Paper',
    description='Material:',
    disabled=False
)
display(material_dropdown)

time_slider = widgets.FloatSlider(value=5, min=0, max=100, step=1, description='Years of Aging:')
uv_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='UV Exposure:')
lux_slider = widgets.FloatSlider(value=50000, min=0, max=100000, step=1000, description='Lux Hours:')
humidity_slider = widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Humidity (%):')
temp_slider = widgets.FloatSlider(value=20, min=-10, max=50, step=1, description='Temperature (°C):')

# Display the sliders
display(time_slider, uv_slider, lux_slider, humidity_slider, temp_slider)

def simulate_exposure_by_material(lab_image, material, exposure_years, uv_exposure, lux_hours, humidity, temperature):
    lab_exposed = lab_image.copy()
    exposure_ratio = exposure_years / 100.0  # Assuming 100 years as a reference

    if material == 'Acidic Paper':
        # Acidic paper tends to yellow and darken over time
        lab_exposed[:, :, 0] -= exposure_ratio * (lab_exposed[:, :, 0] * 0.2)  # Darkening
        lab_exposed[:, :, 2] += exposure_ratio * (lab_exposed[:, :, 2] * 0.3)  # Yellowing
    elif material == 'Alkaline Paper':
        # Alkaline paper is more stable
        lab_exposed[:, :, 0] -= exposure_ratio * (lab_exposed[:, :, 0] * 0.05)
    elif material == 'Chromolithograph Print':
        # Vibrant colors that may fade under UV exposure
        lab_exposed[:, :, 1] -= exposure_ratio * (lab_exposed[:, :, 1] * uv_exposure * 0.1)
        lab_exposed[:, :, 2] -= exposure_ratio * (lab_exposed[:, :, 2] * uv_exposure * 0.1)
    elif material == 'Sanguine Etching':
        # Red pigments that may fade with light exposure
        lab_exposed[:, :, 1] -= exposure_ratio * (lab_exposed[:, :, 1] * lux_hours * 0.00002)
    elif material == 'Steel Engraving':
        # Generally stable but may darken slightly
        lab_exposed[:, :, 0] -= exposure_ratio * (lab_exposed[:, :, 0] * 0.02)
    elif material == 'Wove Paper':
        # May yellow over time
        lab_exposed[:, :, 2] += exposure_ratio * (lab_exposed[:, :, 2] * 0.15)
    elif material == 'Rag Paper':
        # Durable but can yellow slightly
        lab_exposed[:, :, 2] += exposure_ratio * (lab_exposed[:, :, 2] * 0.1)
    else:
        # Default behavior for unknown materials
        lab_exposed[:, :, 0] -= exposure_ratio * (lab_exposed[:, :, 0] * 0.05)
        lab_exposed[:, :, 2] += exposure_ratio * (lab_exposed[:, :, 2] * 0.05)

    # Ensure values stay within valid LAB ranges
    lab_exposed[:, :, 0] = np.clip(lab_exposed[:, :, 0], 0, 100)
    lab_exposed[:, :, 1] = np.clip(lab_exposed[:, :, 1], -128, 127)
    lab_exposed[:, :, 2] = np.clip(lab_exposed[:, :, 2], -128, 127)

    logging.info(f"Simulated exposure for {material} with {exposure_years} years of aging.")
    return lab_exposed

# Added missing function: lab_to_rgb
def lab_to_rgb(lab_image):
    rgb_image = color.lab2rgb(lab_image)
    rgb_image = np.clip(rgb_image, 0, 1)
    rgb_image = (rgb_image * 255).astype(np.uint8)
    return rgb_image

# Added missing function: display_image
def display_image(image, title='Image', save_fig=False, filename=None):
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Image saved as {filename}")
    plt.show()

# Added missing function: apply_fading
def apply_fading(lab_image, predicted_fading):
    lab_faded = lab_image.copy()
    lab_faded[:, :, 0] += predicted_fading[0]
    lab_faded[:, :, 1] += predicted_fading[1]
    lab_faded[:, :, 2] += predicted_fading[2]
    lab_faded[:, :, 0] = np.clip(lab_faded[:, :, 0], 0, 100)
    lab_faded[:, :, 1] = np.clip(lab_faded[:, :, 1], -128, 127)
    lab_faded[:, :, 2] = np.clip(lab_faded[:, :, 2], -128, 127)
    logging.info("Applied predicted fading to the image.")
    return lab_faded

# -----------------------------
# Step 4: Visualization and Delta-E
# -----------------------------

def compute_delta_e(lab1, lab2):
    delta_e = deltaE_ciede2000(lab1, lab2)
    logging.info(f"Delta-E between the two images calculated.")
    return delta_e

def display_color_difference(delta_e, title='Color Difference Map (∆E)', save_fig=False, filename=None):
    plt.figure(figsize=(8, 6))
    plt.imshow(delta_e, cmap='hot')
    plt.colorbar(label='∆E')
    plt.title(title)
    plt.axis('off')
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Color difference map saved as {filename}")
    plt.show()

def plot_histograms(image1, image2, title_suffix='', save_fig=False, filename=None):
    image1_array = np.array(image1)
    image2_array = np.array(image2)
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    colors = ['Red', 'Green', 'Blue']
    for i, color_name in enumerate(colors):
        axs[i].hist(image1_array[..., i].flatten(), bins=256, alpha=0.5, label=f'{color_name} (Original)', color=color_name.lower())
        axs[i].hist(image2_array[..., i].flatten(), bins=256, alpha=0.5, label=f'{color_name} (Faded)', color=f'dark{color_name.lower()}')
        axs[i].set_title(f'{color_name} Channel {title_suffix}')
        axs[i].legend()
    plt.tight_layout()
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Histogram saved as {filename}")
    plt.show()

def display_average_color(image_lab, title='Average Color', save_fig=False, filename=None):
    average_lab = image_lab.mean(axis=(0,1))
    average_rgb = color.lab2rgb(np.reshape(average_lab, (1,1,3))).reshape(1,1,3)
    average_rgb = np.clip(average_rgb, 0, 1)
    plt.figure(figsize=(2,2))
    plt.imshow(np.ones((100,100,3)) * average_rgb)
    plt.title(title)
    plt.axis('off')
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Average color saved as {filename}")
    plt.show()
    logging.info(f"{title}: L={average_lab[0]:.2f}, A={average_lab[1]:.2f}, B={average_lab[2]:.2f}")
    return average_lab

# New function to compute Delta-E between average colors
def compute_average_delta_e(avg_lab1, avg_lab2):
    lab1 = np.array([avg_lab1])
    lab2 = np.array([avg_lab2])
    delta_e = deltaE_ciede2000(lab1, lab2)[0]
    logging.info(f"Delta-E between average colors: {delta_e:.2f}")
    return delta_e

# -----------------------------
# Step 5: Main Execution Flow
# -----------------------------

def main():
    # List of materials
    materials = [
        'Acidic Paper',
        'Alkaline Paper',
        'Chromolithograph Print',
        'Sanguine Etching',
        'Steel Engraving',
        'Wove Paper',
        'Rag Paper',
    ]

    # Upload and load dataset
    csv_filename = upload_file("Please upload your LAB color dataset CSV file.", file_types=('.csv',))
    dataset = load_and_clean_dataset(csv_filename)

    # Upload and process image
    original_image, original_lab = upload_and_process_image()

    # Display average color before fading
    avg_lab_before = display_average_color(original_lab, title='Average Color Before Fading', save_fig=True, filename='average_color_before.png')

    # Create synthetic data and train model
    synthetic_data = create_synthetic_data(materials, num_samples_per_material=500)
    X, Y, encoder = prepare_features(synthetic_data)
    model, scaler, mse = train_ml_model(X, Y)
    print(f"Mean Squared Error for Fading Prediction: {mse:.4f}")

    # Get environmental parameters
    material = material_dropdown.value
    exposure_years = time_slider.value
    uv_exposure = uv_slider.value
    lux_hours = lux_slider.value
    humidity = humidity_slider.value
    temperature = temp_slider.value

    # Simulate exposure by material
    lab_exposed = simulate_exposure_by_material(original_lab, material, exposure_years, uv_exposure, lux_hours, humidity, temperature)
    exposed_image = lab_to_rgb(lab_exposed)
    display_image(exposed_image, title=f'Simulated {material} Exposure ({int(exposure_years)} years)', save_fig=True, filename='exposed_image.png')

    # Prepare features for prediction
    # One-hot encode the selected material
    material_encoded = encoder.transform([[material]])
    X_input = np.hstack(([[lux_hours]], material_encoded))
    X_input_scaled = scaler.transform(X_input)

    # Predict fading and apply it
    predicted_fading = model.predict(X_input_scaled)[0]
    lab_faded = apply_fading(lab_exposed, predicted_fading)
    faded_image = lab_to_rgb(lab_faded)
    display_image(faded_image, title=f'Faded {material} Image After Exposure Prediction', save_fig=True, filename='faded_image.png')

    # Display average color after fading
    avg_lab_after = display_average_color(lab_faded, title='Average Color After Fading', save_fig=True, filename='average_color_after.png')

    # Compute Delta-E between average colors
    delta_e_avg = compute_average_delta_e(avg_lab_before, avg_lab_after)
    print(f"Delta-E between average colors before and after fading: {delta_e_avg:.2f}")

    # Compute pixel-wise Delta-E and display
    delta_e = compute_delta_e(original_lab, lab_faded)
    display_color_difference(delta_e, title='Color Difference Map (∆E)', save_fig=True, filename='delta_e_map.png')
    plot_histograms(original_image, faded_image, title_suffix=f'({material} Exposure)', save_fig=True, filename='histograms.png')

    # Provide download links if running in Google Colab
    try:
        from google.colab import files
        print("Preparing files for download...")
        files_to_download = ['average_color_before.png', 'exposed_image.png', 'faded_image.png', 'average_color_after.png', 'delta_e_map.png', 'histograms.png']
        for file in files_to_download:
            files.download(file)
    except ImportError:
        print("Files saved locally. Download functionality is not available in this environment.")

if __name__ == "__main__":
    main()